# Checking the Velocity Distribution
## particles coordinates obtained from imageJ

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import trackpy as tp
import pandas as pd
import os

In [2]:
folderPath = r'g:\My Drive\workAppState\prj_shocks\expData\data20230530velDistribTest_ImageJ_Connor'

In [3]:
inputsPath = folderPath + r'\inputs'
infilename = r'\360v_1mA_9sccm_91mtorr'

In [4]:
colnames=['x','y','frame']
indf = pd.read_csv(inputsPath + infilename  + r'.csv', usecols=[3,4,8],names=colnames, skiprows=1)


In [5]:
indf.head()

,x,y,frame
0,345.78586,13.41291,1
1,306.32614,14.01396,1
2,332.94084,13.89708,1
3,319.62251,14.91738,1
4,371.09624,14.78872,1


In [6]:
tracked_csv_dir = folderPath + r'\outputs' + r'\tracked' 
tracked_csv_filepath = tracked_csv_dir + infilename + r'_tracked.csv'
if not os.path.exists(tracked_csv_dir):
    os.makedirs(tracked_csv_dir)
    t_large = tp.link(t_csv,3,memory=1)
    t_large.to_csv(tracked_csv_filepath)    
else:
    t_large = pd.read_csv(tracked_csv_filepath)
t_large


,Unnamed: 0,x,y,frame,particle
0,0,345.78586,13.41291,1,0
1,1299,563.45040,319.49206,1,1
2,1298,513.90593,319.84384,1,2
3,1297,284.83552,319.66066,1,3
4,1296,206.13283,319.97588,1,4
...,...,...,...,...,...
1872186,1870898,210.52854,191.02396,968,1920
1872187,1870897,612.83741,188.58557,968,1541
1872188,1870896,494.81270,189.19780,968,1895
1872189,1870864,114.40066,184.15846,968,1864


In [7]:
#function get_velocities(df, step) calculates velocites of particles
# with a given step step.
def get_velocities(df, step):
#initialize empy arrays to store data:
    arr_particle = np.array([])
    arr_x = np.array([])
    arr_y = np.array([])
    arr_vx = np.array([])
    arr_vy = np.array([])
    arr_frame = np.array([])
    # get an array containing all frame numbers in the input dataframe:
    frames_listing = np.unique(np.array(df['frame']))
    #cycle throught all those frames:
    for iFrame in range(step, len(frames_listing)):
        #get current frame:
        cur_frame = frames_listing[iFrame]
        #select a dataframe containing data ONLY for that frame:
        df_front_frame = df[(df['frame'] == cur_frame)]
        print(iFrame)
        #cycle throught all particles in the frame and find their velocities as
        # position of the particle in that frame minus position of the same particles
        # step frames ago:
        for i in range(0, len(df_front_frame)):
            #take i-th particle in a frame
            cur_p = df_front_frame['particle'].iloc[i]
            cur_x = df_front_frame['x'].iloc[i]
            cur_y = df_front_frame['y'].iloc[i]
            #find a row with the same particle in a frame step frames ago:
            prev_frame_cur_row = df[((df['frame'] == cur_frame - step) & (df['particle'] == cur_p))]
            #if that particle excisted back then, we will get exactly ONE row:
            if (len(prev_frame_cur_row) == 1):
                #if this row exists, we can take position of that particle in that, previous, frame:
                prev_x = prev_frame_cur_row['x'].iloc[0]
                prev_y = prev_frame_cur_row['y'].iloc[0]
                # so we can calculate velocities:
                cur_vx = cur_x - prev_x
                cur_vy = cur_y - prev_y
                cur_particle = df_front_frame['particle'].iloc[i]
                #and append all parameters of that particle to our data arrays
                arr_vx = np.append(arr_vx, cur_vx)
                arr_vy = np.append(arr_vy, cur_vy)
                arr_particle = np.append(arr_particle, cur_particle)
                arr_x = np.append(arr_x, cur_x)
                arr_y = np.append(arr_y, cur_y)
                arr_frame = np.append(arr_frame, cur_frame)
    #save output as a dataframe containing all the info we need:
    data = {'frame':arr_frame, 'particle':arr_particle, 'x': arr_x, 'y': arr_y, 'vx': arr_vx, 'vy':arr_vy}
    ret_df = pd.DataFrame(data)
                
    return ret_df

## Getting the Temperature Profiles

#### constants:

In [34]:
step = 25 # number of frames between the frames for calculating velocities
dust_mass = 4.0 / 3.0 * np.pi * (7.14E-6 / 2.0)**3 * 1510.0 #mass of the dust particles
N_points = 15 # number of points in the temperature profile
rate_for_vel = 99.0 / step # frequency for calculating particle speeds
mm_per_px = 1.0/40.3 # camera resolution
N_bins = 20 #number of bins in the hystograms

#### first, get velocities. May take some time:

In [35]:
vel_csv_dir = folderPath + r'\outputs\csvData\velocities'
vel_csv_csv_filepath = vel_csv_dir + infilename + r'_velocities.csv'
if not os.path.exists(vel_csv_dir):
    os.makedirs(vel_csv_dir)
    df_vel = get_velocities(t_large, step)
    df_vel.to_csv(vel_csv_csv_filepath)    
else:
    df_vel = pd.read_csv(vel_csv_csv_filepath)
df_vel.head()

,Unnamed: 0,frame,particle,x,y,vx,vy
0,0,26.0,5.0,87.24011,318.85615,0.41789,-0.31052
1,1,26.0,11.0,390.23453,317.86926,-0.42804,0.49591
2,2,26.0,27.0,501.01584,317.34253,0.25052,0.44268
3,3,26.0,54.0,56.19952,320.86459,0.71546,-2.08758
4,4,26.0,9.0,250.44585,318.53971,-0.02436,0.27801


In [39]:
def get_velocities_fast(df, step):
    df_sorted = df.sort_values(by = ['particle', 'frame'])
    grouped = df_sorted.groupby('particle')
    df_sorted['vx'] = grouped['x'].diff(periods = step)
    df_sorted['vy'] = grouped['y'].diff(periods = step)
    df_sorted = df_sorted.dropna()
    df_sorted = df_sorted.sort_values(by = ['frame', 'particle'])    
    return df_sorted

In [40]:
t_sorted = get_velocities_fast(t_large, step)

In [41]:
len(t_sorted)

1817475

In [42]:
len(df_vel)

1817329

In [43]:
df_vel = df_vel.sort_values(by = ['frame', 'particle'])
df_vel['particle'] = df_vel['particle'].astype('int')
df_vel['frame'] = df_vel['frame'].astype('int')
df_vel.head()

,Unnamed: 0,frame,particle,x,y,vx,vy
1294,1294,26,0,346.28611,13.60750,0.50025,0.19459
42,42,26,1,563.76923,319.58150,0.31883,0.08944
52,52,26,2,514.13874,320.31868,0.23281,0.47484
5,5,26,3,284.52489,320.01674,-0.31063,0.35608
9,9,26,4,205.28021,319.75866,-0.85262,-0.21722


In [44]:
df_vel.tail(10)


,Unnamed: 0,frame,particle,x,y,vx,vy
1816090,1816090,968,2505,574.96842,439.48947,1.30626,0.50974
1815874,1815874,968,2516,25.96001,288.12137,0.71097,-0.57263
1815446,1815446,968,2520,690.48805,322.99004,0.69434,-1.57989
1816083,1816083,968,2527,420.87831,441.82275,1.35188,0.82055
1815501,1815501,968,2529,48.30000,308.74852,-0.95171,-0.44147
1816490,1816490,968,2534,501.72504,149.56259,-0.10174,-0.74679
1815460,1815460,968,2540,651.12000,320.73000,0.70730,-0.78190
1816109,1816109,968,2541,539.22603,453.73288,0.76100,-0.45943
1817169,1817169,968,2545,486.83841,158.18293,-0.64795,-0.33071
1815626,1815626,968,2555,63.06354,336.13928,0.17599,-0.22098


In [45]:
firstColName = t_sorted.columns[0]
#t_sorted = t_sorted.drop(columns = firstColName)
t_sorted.tail(10)


,Unnamed: 0,x,y,frame,particle,vx,vy
1870949,1872072,574.96842,439.48947,968,2505,1.30626,0.50974
1870729,1871389,25.96001,288.12137,968,2516,0.71097,-0.57263
1870295,1871581,690.48805,322.99004,968,2520,0.69434,-1.57989
1870942,1872079,420.87831,441.82275,968,2527,1.35188,0.82055
1870351,1871499,48.30000,308.74852,968,2529,-0.95171,-0.44147
1871351,1870705,501.72504,149.56259,968,2534,-0.10174,-0.74679
1870309,1871567,651.12000,320.73000,968,2540,0.70730,-0.78190
1870968,1872114,539.22603,453.73288,968,2541,0.76100,-0.45943
1872030,1870751,486.83841,158.18293,968,2545,-0.64795,-0.33071
1870477,1871640,63.06354,336.13928,968,2555,0.17599,-0.22098


In [46]:
len(t_sorted)

1817475

In [47]:
len(df_vel)

1817329

In [48]:
t_sorted_temp = t_sorted[['frame', 'particle', 'x']].apply(tuple, axis=1)
df_vel_temp = df_vel[['frame', 'particle', 'x']].apply(tuple, axis=1)

In [49]:
extra_rows = t_sorted[~t_sorted_temp.isin(df_vel_temp)]

In [50]:
extra_rows = extra_rows.sort_values(by = ['particle', 'frame'])
extra_rows.head(20)

,Unnamed: 0,x,y,frame,particle,vx,vy
467376,468542,657.38018,335.68433,242,124,-0.09450,-2.15111
1365124,1366391,659.81875,321.50625,706,124,-0.11335,-0.56165
987775,988928,645.53371,333.51124,511,171,0.34818,-0.02021
1034260,1035410,645.47468,332.74051,535,171,-0.22001,-0.90108
335451,336701,657.12500,324.31250,174,189,0.54113,-0.89073
348990,350293,657.74476,324.26224,181,189,0.77285,-0.74338
362569,363867,657.75806,323.76452,188,189,0.76188,-0.75838
1506128,1507275,660.44211,306.22105,779,189,0.39890,-0.68636
273552,274690,568.42704,332.47425,142,238,0.77257,-0.19648
437770,437125,690.19744,252.55128,226,348,-0.51089,-0.76816


In [51]:
len(extra_rows)

249

In [52]:
get_suku = t_large[(t_large['particle'] == 124) & (t_large['frame'] == 242)]
get_suku.head()

,Unnamed: 0,x,y,frame,particle
467376,468542,657.38018,335.68433,242,124


In [53]:
get_suku_prev = t_large[(t_large['particle'] == 124) & (t_large['frame'] == (242 - step))]
get_suku_prev.head()

,Unnamed: 0,x,y,frame,particle


In [54]:
valid_particles = np.unique(t_large[['particle', 'frame']].values - np.array([0, step]), axis=0)

In [55]:
len(valid_particles)

1872191

In [56]:
particle_time = t_sorted[['particle', 'frame']].values

In [57]:
len(particle_time)

1817475

In [58]:
mask = np.isin(particle_time, valid_particles).all(axis=1)

In [59]:
len(mask)

1817475

In [60]:
t_ultimate = t_sorted[mask]

In [61]:
len(t_ultimate)

1817475

In [62]:
get_suku = t_ultimate[(t_ultimate['particle'] == 171) & (t_ultimate['frame'] == 242)]
get_suku.head()

,Unnamed: 0,x,y,frame,particle,vx,vy
467305,468565,639.47683,340.64286,242,171,0.93366,0.28674


In [63]:
get_suku_prev = t_large[(t_large['particle'] == 124) & (t_large['frame'] == (216))]
get_suku_prev.head()

,Unnamed: 0,x,y,frame,particle
417024,418178,657.47468,337.83544,216,124


In [64]:
ind_particle = t_sorted[(t_sorted['particle'] == 124)]
ind_particle.head(10)

,Unnamed: 0,x,y,frame,particle,vx,vy
48556,49850,655.95536,343.38988,26,124,0.76710,-0.21943
50493,51787,655.95486,343.51389,27,124,0.77870,-0.45230
52430,53725,655.97751,343.52422,28,124,0.68318,-0.49706
54368,55661,655.99615,343.46538,29,124,0.48573,-0.30893
56301,57600,656.00424,343.40678,30,124,0.43081,-0.21909
58246,59533,656.09930,343.05749,31,124,0.55518,-0.53075
60181,61473,655.98110,342.87113,32,124,0.48913,-0.69313
62121,63411,655.90081,342.79150,33,124,0.47503,-0.73975
64064,65348,655.99813,342.75094,34,124,0.60924,-0.77415
65997,67289,656.09289,342.69763,35,124,0.65411,-0.83094


In [65]:
np.max(ind_particle['frame'])

968

In [66]:
ind_particle.tail(10)

,Unnamed: 0,x,y,frame,particle,vx,vy
1852936,1854182,662.25225,312.36486,959,124,0.67458,-0.64485
1854862,1856109,662.27778,312.30769,960,124,0.62950,-0.53714
1856883,1858031,662.29623,312.12642,961,124,0.73764,-0.51030
1858783,1859958,662.31818,312.10474,962,124,0.82823,-0.71184
1860681,1861886,662.28571,312.01429,963,124,0.86811,-0.84901
1862607,1863816,662.33406,311.77074,964,124,0.86771,-1.07541
1864500,1865744,662.36614,311.74016,965,124,1.02874,-1.00374
1866521,1867672,662.40566,311.71132,966,124,1.09513,-1.03429
1868450,1869600,662.42086,311.69784,967,124,0.98373,-1.03569
1870321,1871527,662.43701,311.69685,968,124,0.94042,-0.87824


In [67]:
indiv_frames = np.array(ind_particle['frame'])
indiv_frames

array([ 26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 20